<a href="https://colab.research.google.com/github/sohomghosh/Finsim4_ESG/blob/main/FinSim4_ESG_model_make.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The token in the next line is to be removed

In [1]:
!rm -rf /content/Finsim4_ESG

In [2]:
!git clone https://sohomghosh:<token_id>@github.com/sohomghosh/Finsim4_ESG.git

Cloning into 'Finsim4_ESG'...
remote: Enumerating objects: 240, done.
remote: Counting objects: 100% (240/240), done.
remote: Compressing objects: 100% (230/230), done.
remote: Total 240 (delta 18), reused 213 (delta 5), pack-reused 0
Receiving objects: 100% (240/240), 13.53 MiB | 15.21 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [3]:
!pip install sentence_transformers

     |████████████████████████████████| 79 kB 3.5 MB/s 
     |████████████████████████████████| 4.2 MB 10.5 MB/s 
     |████████████████████████████████| 1.2 MB 1.5 MB/s 
     |████████████████████████████████| 84 kB 2.9 MB/s 
     |████████████████████████████████| 596 kB 44.5 MB/s 
     |████████████████████████████████| 6.6 MB 43.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=8959f4272263f6cbd7a08af5d2806498d781744f19423b59637f8e69dfd068f4
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import numpy as np
import json
from sklearn.metrics import accuracy_score
from sentence_transformers import SentenceTransformer, losses, SentencesDataset, evaluation, InputExample, util
from torch.utils.data import DataLoader
import torch

In [5]:
import pandas as pd

In [6]:
import random

In [ ]:
!pwd

/content


In [7]:
path = '/content/Finsim4_ESG/data/'

In [8]:
def compute_average_rank(Y_test, predictions):
  ranks = []
  for idx, y in enumerate(Y_test):
    ranks.append(get_rank(Y_test[idx], predictions[idx]))
  return np.mean(ranks)

def get_rank(gold, list_predictions, max_k=3):
  list_predictions = list_predictions[:max_k]
  try:
    rank = list_predictions.index(gold) + 1
  except ValueError:
    rank = max_k + 1
  return rank

In [9]:
task1_df = pd.read_json(path + '/terms/ESG_taxonomy_train.json')[['term', 'concept']]
task1_df

,term,concept
0,low-carbon,Carbon factor
1,carbon footprint,Carbon factor
2,lower-carbon economy,Carbon factor
3,carbon data,Carbon factor
4,carbon values,Carbon factor
...,...,...
642,Shareholders Approval Stock Compensation Plan,Executive compensation
643,Mgt Compensation Controversies Count,Executive compensation
644,Recent Mgt Compensation Controversies,Executive compensation
645,Fair & Transparent Remuneration,SHARE CAPITAL


In [ ]:
task1_df.shape

(647, 2)

In [ ]:
task1_df['concept'].value_counts()

Energy efficiency and renewable energy                          59
Sustainable Food & Agriculture                                  54
Product Responsibility                                          51
circular economy                                                47
Sustainable Transport                                           46
Emissions                                                       39
Shareholder rights                                              38
Board Make-Up                                                   37
Injury frequency rate for subcontracted labour                  35
Executive compensation                                          32
Biodiversity                                                    29
Community                                                       27
Employee engagement                                             23
Employee development                                            22
Water & waste-water management                                

In [ ]:
task1_df['concept'].value_counts(normalize = True)

Energy efficiency and renewable energy                          0.091190
Sustainable Food & Agriculture                                  0.083462
Product Responsibility                                          0.078825
circular economy                                                0.072643
Sustainable Transport                                           0.071097
Emissions                                                       0.060278
Shareholder rights                                              0.058733
Board Make-Up                                                   0.057187
Injury frequency rate for subcontracted labour                  0.054096
Executive compensation                                          0.049459
Biodiversity                                                    0.044822
Community                                                       0.041731
Employee engagement                                             0.035549
Employee development                               

In [ ]:
task2_df = pd.read_json(path + '/sentences/Sustainability_sentences_train.json')
task2_df

,sentence,label
0,"For example, our portfolio companies have set ...",sustainable
1,We continue to increase the percentage of rene...,sustainable
2,"Acea Ambiente, the Group Company operating in ...",sustainable
3,Carbon intensity is measured as tons of CO2 pe...,unsustainable
4,"More circular, efficient production and transp...",unsustainable
...,...,...
2260,"We anticipate that the initiative, which was l...",sustainable
2261,The analytics of decarbonization demand a coll...,unsustainable
2262,BlackRock achieved its 100% renewable electric...,sustainable
2263,"Beyond carbon intensity, SCOR believes that de...",unsustainable


In [ ]:
task2_df.shape

(2265, 2)

In [ ]:
task2_df['label'].value_counts()

sustainable      1223
unsustainable    1042
Name: label, dtype: int64

In [ ]:
task2_df['label'].value_counts(normalize = True)

sustainable      0.539956
unsustainable    0.460044
Name: label, dtype: float64

In [10]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("Rodion/sbert_uno_sustainable_development_goals")

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/723 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/470 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [11]:
concept_defn = pd.read_csv(path+"concept_definition.csv")
concept_defn

,CONCEPT,DEFINITION
0,Carbon factor,Carbon factor is ‘the factor that is applied t...
1,Emissions,Emissions are substances that are emitted;[2] ...
2,Energy efficiency and renewable energy,Energy efficiency is the use of less energy to...
3,Waste management,Waste management (or waste disposal) includes ...
4,Water & waste-water management,The goal of wastewater management is to clean ...
5,Biodiversity,Biodiversity is the biological variety and var...
6,Sustainable Transport,Sustainable transport refers to ways of transp...
7,Sustainable Food & Agriculture,Sustainable agriculture is farming in sustaina...
8,circular economy,"A circular economy is ""a model of production a..."
9,Employee development,Employee development is a process of improving...


In [12]:
concept_defn_di = {}
for i,j in zip(concept_defn['CONCEPT'].values, concept_defn['DEFINITION'].values):
  concept_defn_di[i] = j

In [ ]:
concept_defn_di

{'Audit Oversight': 'Audit oversight refers to the verification and checking of the reported financial results of an organization by external and internal review.',
 'Biodiversity': 'Biodiversity is the biological variety and variability of life on Earth. Biodiversity is a measure of variation at the genetic, species, and ecosystem level. Terrestrial biodiversity is usually greater near the equator, which is the result of the warm climate and high primary productivity',
 'Board Independence': 'Independence occurs when a board member has not been and is not currently employed by the company or its auditor and the board member’s employer doesn’t do a significant amount of business with the company.',
 'Board Make-Up': 'Governing body of a company, elected by shareholders to set strategy and oversee management',
 'Carbon factor': 'Carbon factor is ‘the factor that is applied to electricity that is consumed by buildings, to understand that carbon emissions associated with the electricity u

In [13]:
task1_con_defn_df = task1_df.merge(concept_defn, left_on='concept', right_on='CONCEPT').drop('CONCEPT', axis = 1)
task1_con_defn_df

,term,concept,DEFINITION
0,low-carbon,Carbon factor,Carbon factor is ‘the factor that is applied t...
1,carbon footprint,Carbon factor,Carbon factor is ‘the factor that is applied t...
2,lower-carbon economy,Carbon factor,Carbon factor is ‘the factor that is applied t...
3,carbon data,Carbon factor,Carbon factor is ‘the factor that is applied t...
4,carbon values,Carbon factor,Carbon factor is ‘the factor that is applied t...
...,...,...,...
642,Shareholders Approval Stock Compensation Plan,Executive compensation,Executive compensation is composed of both the...
643,Mgt Compensation Controversies Count,Executive compensation,Executive compensation is composed of both the...
644,Recent Mgt Compensation Controversies,Executive compensation,Executive compensation is composed of both the...
645,Fair & Transparent Remuneration,SHARE CAPITAL,"A corporation's share capital, commonly referr..."


In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_df, val_df = train_test_split(task1_con_defn_df, test_size=0.2, random_state=42)

In [ ]:
train_df

,term,concept,DEFINITION
332,Hours of training\nto develop digital skills,Employee development,Employee development is a process of improving...
29,energy transition,Emissions,Emissions are substances that are emitted;[2] ...
302,product’s lifecycle,circular economy,"A circular economy is ""a model of production a..."
286,chemical recycling of plastic waste,circular economy,"A circular economy is ""a model of production a..."
417,Employee Health & Safety Training Hours,Injury frequency rate for subcontracted labour,"The number of injuries per 1,000,000 employee-..."
...,...,...,...
71,Environmental Project Financing,Energy efficiency and renewable energy,Energy efficiency is the use of less energy to...
106,Renewable Energy Produced,Energy efficiency and renewable energy,Energy efficiency is the use of less energy to...
270,seed,Sustainable Food & Agriculture,Sustainable agriculture is farming in sustaina...
435,Contractor Lost working Days,Injury frequency rate for subcontracted labour,"The number of injuries per 1,000,000 employee-..."


In [ ]:
val_df

,term,concept,DEFINITION
634,Policy Executive Compensation Performance,Executive compensation,Executive compensation is composed of both the...
220,Transportation Stewardship,Sustainable Transport,Sustainable transport refers to ways of transp...
426,Employee Accidents,Injury frequency rate for subcontracted labour,"The number of injuries per 1,000,000 employee-..."
428,Employee Fatalities,Injury frequency rate for subcontracted labour,"The number of injuries per 1,000,000 employee-..."
72,Nuclear,Energy efficiency and renewable energy,Energy efficiency is the use of less energy to...
...,...,...,...
513,Recent Customer Health & Safety Controversies,Product Responsibility,product responsibility is the need to supply h...
374,develop their careers,Future of work,The future of work describes changes in how wo...
443,Policy Business Ethics,Community,A community is a social unit with commonality ...
244,food delivery,Sustainable Food & Agriculture,Sustainable agriculture is farming in sustaina...


# Negative Sample Creation

In [16]:
all_concepts_set = set(task1_df['concept'].values)

In [17]:
random.seed(0)

In [18]:
sent1, sent2, sim12 = [], [], []

for term, concept, concept_defn_txt in zip(train_df['term'].values, train_df['concept'].values, train_df['DEFINITION'].values):
  sent1.append(term)
  sent2.append(concept_defn_txt)
  sim12.append(1.0)

  remaining_concept = list(all_concepts_set - set([concept]))
  random_index = random.randint(0,len(remaining_concept)-1)
  random_concept = remaining_concept[random_index]

  sent1.append(term)
  sent2.append(concept_defn_di[random_concept])
  sim12.append(0.0) 

In [19]:
full_df = pd.DataFrame({'term':sent1, 'concept':sent2, 'sim':sim12})
full_df

,term,concept,sim
0,Hours of training\nto develop digital skills,Employee development is a process of improving...,1.0
1,Hours of training\nto develop digital skills,Employee Recruitment alludes to the overall pr...,0.0
2,energy transition,Emissions are substances that are emitted;[2] ...,1.0
3,energy transition,Employee Recruitment alludes to the overall pr...,0.0
4,product’s lifecycle,"A circular economy is ""a model of production a...",1.0
...,...,...,...
1029,seed,Employee engagement is a fundamental concept i...,0.0
1030,Contractor Lost working Days,"The number of injuries per 1,000,000 employee-...",1.0
1031,Contractor Lost working Days,Independence occurs when a board member has no...,0.0
1032,Energy Produced Direct,Energy efficiency is the use of less energy to...,1.0


In [20]:
model1 = SentenceTransformer("Rodion/sbert_uno_sustainable_development_goals")
model_save_path = 'sbert_uno_sustainable_development_goals_pos_neg_NEW'
train_batch_size = 20
num_epochs = 15

distance_metric = losses.SiameseDistanceMetric.COSINE_DISTANCE
margin = 0.5

train_samples_MultipleNegativesRankingLoss = []
train_samples_ConstrativeLoss = []

for sent1,sent2,sim in zip(full_df['term'].values, full_df['concept'].values, full_df['sim'].values):
    train_samples_ConstrativeLoss.append(InputExample(texts=[sent1, sent2], label=sim))
    if sim == 1.0:
        train_samples_MultipleNegativesRankingLoss.append(InputExample(texts=[sent1, sent2], label=sim))
        train_samples_MultipleNegativesRankingLoss.append(InputExample(texts=[sent1, sent2], label=sim))  # if A is a duplicate of B, then B is a duplicate of A

# Create data loader and loss for MultipleNegativesRankingLoss
train_dataset_MultipleNegativesRankingLoss = SentencesDataset(train_samples_MultipleNegativesRankingLoss, model=model1)
train_dataloader_MultipleNegativesRankingLoss = DataLoader(train_dataset_MultipleNegativesRankingLoss, shuffle=True, batch_size=train_batch_size)
train_loss_MultipleNegativesRankingLoss = losses.MultipleNegativesRankingLoss(model1)


# Create data loader and loss for OnlineContrastiveLoss
train_dataset_ConstrativeLoss = SentencesDataset(train_samples_ConstrativeLoss, model=model1)
train_dataloader_ConstrativeLoss = DataLoader(train_dataset_ConstrativeLoss, shuffle=True, batch_size=train_batch_size)
train_loss_ConstrativeLoss = losses.OnlineContrastiveLoss(model=model1, distance_metric=distance_metric, margin=margin)



In [ ]:
# .....
# Train the model
model1.fit(train_objectives=[(train_dataloader_MultipleNegativesRankingLoss, train_loss_MultipleNegativesRankingLoss), (train_dataloader_ConstrativeLoss, train_loss_ConstrativeLoss)],
          #evaluator=seq_evaluator,
          epochs=num_epochs,
          warmup_steps=1000,
          output_path=model_save_path
          )

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Iteration:   0%|          | 0/52 [00:00<?, ?it/s]

In [ ]:
corpus = list(all_concepts_set)
N = 25

In [ ]:
corpus_id_di = {}
for i in enumerate(corpus):
  corpus_id_di[i[1]]=i[0]

In [ ]:
def results_pred(model, df):

  corpus_embedding = model.encode(corpus, convert_to_tensor=True)
  # Query sentences:
  queries = list(df['term'].values)

  top_k = min(N, len(corpus))
  all_hits = []
  for query in queries:
    query_embedding = model.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(query_embedding, corpus_embedding, top_k=N)
    all_hits.append(hits)

  lli = []
  for kk in all_hits:
    kk_trns = {}
    for i in kk[0]:
      kk_trns[i['corpus_id']] = i['score']
    for i in range(N):
      if i not in kk_trns.keys():
        kk_trns[i] = 0.0
    
    lli.append([kk_trns[i] for i in range(N)])
  

  valid_y = list(df['concept'].values)

  predicted_df = pd.DataFrame({'term' : list(df['term'].values), 'concept':valid_y})
  predicted_df = pd.concat([predicted_df, pd.DataFrame(lli)], axis = 1)

  pddf = predicted_df.groupby(['term', 'concept']).mean().reset_index()
  pddf['li'] = pddf[[i for i in range(N)]].values.tolist()

  pddf['pred_valid'] = [np.argmax(i) for i in pddf['li'].values]
  pddf['predictions_to_rank'] = [list(np.argsort(i)[::-1]) for  i in pddf['li'].values]
  pddf['concept_mapped'] = pddf['concept'].apply(lambda x : corpus_id_di[x])
  print(compute_average_rank(list(pddf['concept_mapped']),list(pddf['predictions_to_rank'])), accuracy_score(list(pddf['concept_mapped']), list(pddf['pred_valid'])))
  return pddf

In [ ]:
results_v1 = results_pred(model1, val_df)

In [ ]:
# uno_bert fine tune on the given corpus using MLM

In [ ]:
# intelligently negative sample creation (hierarchy / similar cluster)

In [ ]:
# more epoch, more sample size try

In [ ]:
# think of classification model as well

In [ ]:
# task-2 : FineTune [CLS] token for classification... use base embedding as the one generated from step-

# fine-tuned embedding from task-1 + ML (LR/RF/LR/XG-Boost/Light-GBM)

#generic non-fined tuned embedding

In [ ]:
#[after] data augmentation <for terms>

In [ ]:
#done# train and validation set create

#done, performance deteriorated# also add concept with concept definition

#done, performance deteriorated# only keep concept (not definition)

Results


all-mpnet-base-v2: batch_size 20, epoch 15, only positive
(1.4692307692307693 0.6923076923076923)


Rodion/sbert_uno_sustainable_development_goals: batch_size 20, epoch 15, only positive
(1.5307692307692307 0.676923076923077)


all-mpnet-base-v2: batch_size 20, epoch 15, positive/negative
(1.5769230769230769 0.7)


Rodion/sbert_uno_sustainable_development_goals: batch_size 20, epoch 15,  positive/negative
(1.4230769230769231 0.7692307692307693)


Rodion/sbert_uno_sustainable_development_goals: batch_size 20, epoch 15,  positive/negative/'concept' as well with definition
(1.4615384615384615 0.7153846153846154)


Rodion/sbert_uno_sustainable_development_goals: batch_size 20, epoch 15,  positive/negative/'concept' only <no definition>
(1.4846153846153847 0.7461538461538462)